# 作業場所をAWS上に構築

## ユーザ名を識別子として保存

In [ ]:
pwd = !pwd
username = pwd[0].split("/")[2]
username

## awscliの確認

In [ ]:
!aws --version

## 認証情報の設定

In [ ]:
target_region = 'us-west-2'
!aws configure set default.region {target_region}
!aws configure set default.output json

このノートブックをEC2インスタンス上で実行していて、かつ、EC2インスタンスに必要なAPI発行可能なロールが付与されている場合、以下のKey設定手順は不要です。

----

認証情報としてAccessKeyIdとSecretAccessKeyを設定します。

コマンドに対する対話的な操作が必要ですから、JupyterのTerminal(treeページの[New] - [Terminal]から選択できます)から、 `aws configure` を実施してください。

`aws configure` の実施例:

```
$ aws configure
AWS Access Key ID [None]: (自身のアカウントのアクセスキー)
AWS Secret Access Key [None]: (自身のアカウントのシークレットアクセスキー)
Default region name [us-west-2]: (Enter)
Default output format [json]: (Enter)
```


## keypairの作成

クラウド上でLinuxマシンを起動する前に、ssh接続に必要となるkeypairを作成します。

In [ ]:
!aws ec2 delete-key-pair --key-name {username}
row = !aws ec2 create-key-pair --key-name {username} \
        --query 'KeyMaterial' --output text

with open(username + '.pem', mode='w') as f:
    f.write('\n'.join(row))

import os
os.chmod(username + '.pem', 0o600)

## 仮想マシンの起動

ansibleでec2インスタンスを起動します。ディストリビューションはCentOS7とします。

マーケットプレイスでCentOS7のAMIを調べます。

In [ ]:
!aws ec2 describe-images --owners aws-marketplace \
--filters Name=product-code,Values=aw0evgkw8e5c1q413zgy5pjce \
--query 'reverse(sort_by(Images[].\
{ami:ImageId, date:CreationDate, text:Description}, &date))'

セキュリティグループを調べます。

In [ ]:
!aws ec2 describe-security-groups --query 'sort_by(SecurityGroups[].\
{name:GroupName, id:GroupId, vpc:VpcId}, &name)'

サブネットを調べます。

In [ ]:
!aws ec2 describe-subnets --query 'sort_by(Subnets[].\
{id:SubnetId, vpc:VpcId, az:AvailabilityZone}, &az)'

t3a.mediumでスポットインスタンスを起動します。

ansibleで起動するためのplaybookを書きます。

In [ ]:
image = 'ami-0bc06212a56393ee1' # CentOS7
instance_type = 't3a.medium'
subnet = 'subnet-916c37ba' # 先に作成済みのものを先の結果から引き写す
sg = 'default' # 先に作成済みのものを先の結果から引き写す

content = f'''
---
- hosts: localhost
  gather_facts: no
  tasks:
  - name: "create ec2 instance"
    amazon.aws.ec2:
      region: {target_region}
      key_name: {username}
      instance_type: {instance_type}
      image: {image}
      group: {sg}
      vpc_subnet_id: {subnet}
      wait: yes
      assign_public_ip: yes
      spot_price: "0.9"
      instance_initiated_shutdown_behavior: terminate
      volumes:
        - device_name: /dev/sda1
          volume_type: gp2
          volume_size: 32
          delete_on_termination: true
      instance_tags:
        Name: {username}_docker_spot
        Owner: {username}
#      profile: default # インスタンスロールが効いてる場合は不要
    register: ec2

  - name: "return status"
    debug:
      msg: "{{{{ ec2 }}}}"
'''.strip()

with open('create_ec2_instance.yml', 'w') as f:
    print(content, file=f)

!cat create_ec2_instance.yml

起動。

In [ ]:
row = !ansible-playbook create_ec2_instance.yml

import json

while row[0].find('    "msg": {') == -1:
    del row[0]
row[0] = "{"
c = 0
for line in row :
    c += 1
    if line.find('    }') == 0:
        break
row = row[:c]
linux = json.loads(''.join(row))
print(linux)

起動が完了するまで待ちます。

In [ ]:
!aws ec2 wait instance-status-ok --include-all-instances --instance-ids {linux['instance_ids'][0]}

ansibleの接続情報を作成します。

インスタンスがJupyterと同じVPC内に起動している場合、接続先にはprivate_ipを指定します。そうでない場合は、Jupyterから接続できるようにセキュリティグループを設定の上、public_ipを指定します。

In [ ]:
host = linux["instances"][0]["private_ip"]
ansible_user = "centos"
ansible_ssh_private_key_file = username + ".pem"

with open('hosts', mode='w') as f:
    f.write(host + ' ansible_user=' + ansible_user + ' ansible_ssh_private_key_file='
            + ansible_ssh_private_key_file)

作成した接続情報を使う前に、対象となるLinuxマシンを信頼できるホストとして、fingerprint登録しておきます。JupyterのTerminalを使って、ダミーのssh実行を行うと登録されます。

In [ ]:
pwd = !pwd
print("ssh -i "+pwd[0]+"/"+ansible_ssh_private_key_file+" "+ansible_user+"@"+host)

```
$ ssh -i xxxx.pem centos@（対象となるLinuxマシンのIPアドレス）
（中略）
Are you sure you want to continue connecting (yes/no)? yes←信頼しますよの意味で入力する
（以下略）
```

として、fingerprint登録します。ssh接続は
```
$ exit
```
で終了させてください。

実務では、ansibleの対象となるLinuxマシンが大量だったり不定だったりするかも知れません。その場合は、sshやansibleの設定で、fingerprint登録の有無を確認しないこともできます。

人手の作業を見落とさないように、必ず失敗するセルを置いておきます。

In [ ]:
raise Exception

ansibleでyum updateが動くことを確認します。

In [ ]:
!ansible -i hosts $host -m yum -a "name=* state=latest" --become

## Docker環境を構築

https://docs.docker.com/engine/install/centos/

公式手順を使って構築します。

In [ ]:
!ansible -i hosts $host -m yum -a "name=docker,docker-client,docker-client-latest,docker-common,docker-latest,docker-latest-logrotate,docker-logrotate,docker-engine \
state=absent autoremove=yes" --become

In [ ]:
!ansible -i hosts $host -m yum -a "name=yum-utils state=latest" --become

In [ ]:
!ansible -i hosts $host -a "yum-config-manager --add-repo \
https://download.docker.com/linux/centos/docker-ce.repo" --become

In [ ]:
!ansible -i hosts $host -m yum -a "name=docker-ce,docker-ce-cli,containerd.io \
state=latest" --become

In [ ]:
!ansible -i hosts $host -m service \
-a "name=docker state=started enabled=yes" --become

In [ ]:
!ansible -i hosts $host -a "docker --version" --become

In [ ]:
!ansible -i hosts $host -a "docker ps -a" --become

docker-composeをインストールします。

https://docs.docker.com/compose/install/

In [ ]:
unames = !ansible -i hosts $host -a "uname -s"

In [ ]:
unamem = !ansible -i hosts $host -a "uname -m"

In [ ]:
urlargs = "\"url=https://github.com/docker/compose/releases/download/1.27.4/\
docker-compose-"+unames[1]+"-"+unamem[1]\
+" dest=/usr/local/bin/docker-compose mode=0555\""

In [ ]:
!ansible -i hosts $host -m get_url -a $urlargs --become

In [ ]:
!ansible -i hosts $host -m file -a "src=/usr/local/bin/docker-compose \
dest=/usr/bin/docker-compose state=link" --become

In [ ]:
!ansible -i hosts $host -a "docker-compose version" --become

## sshカーネル用の設定ファイルを作成

In [ ]:
public_ip = linux['instances'][0]['public_ip']
!echo "Host target" > ~/.ssh/config
!echo "  Hostname $host" >> ~/.ssh/config
!echo "  User $ansible_user" >> ~/.ssh/config
!echo "  Port 22" >> ~/.ssh/config
!echo "  IdentityFile $(pwd)/$ansible_ssh_private_key_file" >> ~/.ssh/config
!echo "#public_ip $public_ip" >> ~/.ssh/config
!cat ~/.ssh/config

## 仮想マシンの破棄

誤って実行してしまわないように、必ず失敗するセルを置いておきます。

In [ ]:
raise Exception

In [ ]:
!aws ec2 terminate-instances --instance-ids {linux['instance_ids'][0]}